<a href="https://colab.research.google.com/github/thomaslu678/gee-test/blob/main/clean/3_clean_export_sort_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTE: Requires landsat_lst_city_points.csv (list of LST for all points across all time)

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from datetime import timedelta
import scipy.stats as stats
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize
import geopandas as gpd
from shapely.geometry import Point
import requests

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read new CSV

In [3]:
export_new_df = pd.read_csv('/content/drive/MyDrive/Year 2/Fall 2025/HONOR 3700/Data/2/landsat_city_points_minimal.csv')

In [4]:
export_new_df

,point_id,mission,time,Blue,Red,NIR,SWIR1,SWIR2,LST_K
0,2505,L4,1988-12-24 01:43:00,0.195835,0.210300,0.213737,0.196330,0.176255,272.622947
1,2527,L4,1988-12-24 01:43:00,0.202545,0.210300,0.213737,0.172900,0.148782,272.622947
2,2528,L4,1988-12-24 01:43:00,0.202545,0.204415,0.199850,0.163550,0.135087,272.622947
3,2550,L4,1988-12-24 01:43:00,0.195835,0.204415,0.192865,0.168225,0.141935,272.718652
4,2551,L4,1988-12-24 01:43:00,0.195835,0.210300,0.213737,0.177575,0.148782,272.718652
...,...,...,...,...,...,...,...,...,...
2646549,4618,L9,2026-01-31 02:11:17,0.065045,0.089052,0.131595,0.151615,0.115177,274.547293
2646550,4619,L9,2026-01-31 02:11:17,0.069005,0.108495,0.204250,0.236617,0.178015,274.506276
2646551,4620,L9,2026-01-31 02:11:17,0.117405,0.163055,0.224133,0.170673,0.129010,274.725030
2646552,4627,L9,2026-01-31 02:11:17,0.060067,0.080857,0.111052,0.118175,0.096917,274.588309


In [5]:
b = export_new_df['Blue']
r = export_new_df['Red']
nir = export_new_df['NIR']
swir1 = export_new_df['SWIR1']
swir2 = export_new_df['SWIR2']
lst_k = export_new_df['LST_K']

In [17]:
columns = ['point_id','time']
export_clean_df = export_new_df[columns]

In [18]:
export_clean_df['LST_C'] = lst_k - 273.15
export_clean_df['NDVI'] = (nir - r) / (nir + r)
export_clean_df['NDBI'] = (swir1 - nir) / (swir1 + nir)
export_clean_df['NDMI'] = (nir - swir1) / (nir + swir1)
export_clean_df['Albedo'] = ((0.356*b)+(0.130*r)+(0.373*nir)+(0.085*swir1)+(0.072*swir2)- 0.018)/ 1.016

/tmp/ipython-input-1748068831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  export_clean_df['LST_C'] = lst_k - 273.15
/tmp/ipython-input-1748068831.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  export_clean_df['NDVI'] = (nir - r) / (nir + r)
/tmp/ipython-input-1748068831.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [19]:
export_clean_df

,point_id,time,LST_C,NDVI,NDBI,NDMI,Albedo
0,2505,1988-12-24 01:43:00,-0.527053,0.008107,-0.042450,0.042450,0.185196
1,2527,1988-12-24 01:43:00,-0.527053,0.008107,-0.105622,0.105622,0.183640
2,2528,1988-12-24 01:43:00,-0.527053,-0.011292,-0.099890,0.099890,0.176036
3,2550,1988-12-24 01:43:00,-0.431348,-0.029073,-0.068238,0.068238,0.171996
4,2551,1988-12-24 01:43:00,-0.431348,0.008107,-0.092413,0.092413,0.181680
...,...,...,...,...,...,...,...
2646549,4618,2026-01-31 02:11:17,1.397293,0.192808,0.070690,-0.070690,0.085628
2646550,4619,2026-01-31 02:11:17,1.356276,0.306176,0.073418,-0.073418,0.127741
2646551,4620,2026-01-31 02:11:17,1.575030,0.157747,-0.135409,0.135409,0.149991
2646552,4627,2026-01-31 02:11:17,1.438309,0.157339,0.031072,-0.031072,0.071202


In [20]:
export_clean_df.to_csv("/content/sample_data/export_clean_calculations.csv", index=False)

# Reading Export CSV

In [ ]:
export_csv = pd.read_csv("/content/sample_data/landsat_lst_city_points.csv")

In [ ]:
len(export_csv)

2644644

In [ ]:
export_csv[0:5]

,system:index,LST_K,distance,image_id,lat,long,mission,time,.geo
0,0_LT04_116034_19881224_2504_0,272.622947,317.553292,LT04_116034_19881224,37.572606,127.013452,L4,1988-12-24 01:43:00,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,0_LT04_116034_19881224_2526_0,272.622947,317.478476,LT04_116034_19881224,37.572608,127.013792,L4,1988-12-24 01:43:00,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,0_LT04_116034_19881224_2527_0,272.622947,287.478569,LT04_116034_19881224,37.572337,127.013794,L4,1988-12-24 01:43:00,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,0_LT04_116034_19881224_2549_0,272.718652,287.403753,LT04_116034_19881224,37.572339,127.014134,L4,1988-12-24 01:43:00,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,0_LT04_116034_19881224_2550_0,272.718652,257.403847,LT04_116034_19881224,37.572068,127.014135,L4,1988-12-24 01:43:00,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [ ]:
export_csv["distance"].nunique()

4628

In [ ]:
export_csv.columns

Index(['system:index', 'LST_K', 'distance', 'image_id', 'lat', 'long',
       'mission', 'time', '.geo'],
      dtype='object')

In [ ]:
export_csv["distance"].value_counts()

,count
distance,
288.059684,674
320.772371,674
256.290609,671
226.346148,669
286.235071,669
...,...
95.446411,465
211.468217,464
215.114519,463


In [ ]:
export_csv["distance"][0:5]

,distance
0,317.553292
1,317.478476
2,287.478569
3,287.403753
4,257.403847


In [ ]:
export_csv['distance'][0]

np.float64(317.553291770426)

In [ ]:
np.where(export_csv['distance'] == np.float64(317.553291770426))[0].tolist()

[0,
 2274,
 6790,
 11418,
 16046,
 20864,
 26689,
 33350,
 37080,
 44957,
 49851,
 55410,
 59208,
 63235,
 67851,
 70765,
 74458,
 79596,
 85911,
 89348,
 93159,
 97566,
 102194,
 106265,
 110218,
 113851,
 119594,
 124207,
 128835,
 133463,
 138091,
 141771,
 145906,
 150534,
 154154,
 156469,
 159070,
 163041,
 167661,
 172459,
 177087,
 181703,
 186099,
 190281,
 194520,
 199103,
 203717,
 208345,
 211510,
 214764,
 218361,
 220325,
 223065,
 227565,
 232193,
 238270,
 245966,
 250510,
 252943,
 255449,
 261855,
 269154,
 273782,
 278213,
 283442,
 288023,
 291725,
 293488,
 300530,
 305022,
 309414,
 313609,
 314343,
 316736,
 321049,
 325677,
 329212,
 333459,
 338086,
 342714,
 347342,
 351268,
 354944,
 359572,
 363024,
 366801,
 370070,
 374826,
 379454,
 383934,
 388245,
 396793,
 400759,
 404008,
 407172,
 410356,
 413212,
 415913,
 419554,
 423179,
 425462,
 428925,
 434292,
 438811,
 442881,
 445792,
 448881,
 453491,
 458110,
 462738,
 470133,
 474631,
 479259,
 484298,
 4

# In copy, group the df by lat/long pair coordinates, sort each group by time, add year

In [ ]:
export_csv_copy = export_csv.copy()

In [ ]:
export_csv_copy.groupby(['lat','long']).size().reset_index().rename(columns={0:'count'})

,lat,long,count
0,37.565193,126.987004,531
1,37.565195,126.987344,537
2,37.565196,126.987684,512
3,37.565198,126.988023,512
4,37.565199,126.988363,512
...,...,...,...
4623,37.574877,127.040277,595
4624,37.574878,127.040617,606
4625,37.574879,127.040957,558
4626,37.574881,127.041296,605


In [ ]:
export_csv_copy['time'] = pd.to_datetime(export_csv_copy['time'])

In [ ]:
export_csv_copy = export_csv_copy.sort_values(
    by=['lat', 'long', 'time'],
    ascending=[True, True, True]
).reset_index(drop=True)

In [ ]:
export_csv_copy.groupby(['lat', 'long'])['time'].is_monotonic_increasing.all()

np.True_

In [ ]:
export_csv_copy['time'] = pd.to_datetime(export_csv_copy['time'])
export_csv_copy['year'] = export_csv_copy['time'].dt.year

In [ ]:
export_csv_copy

,system:index,LST_K,distance,image_id,lat,long,mission,time,.geo,year
0,1_LT05_116034_19840730_797_0,314.209997,326.113118,LT05_116034_19840730,37.565193,126.987004,L5,1984-07-30 01:40:17,"{""type"":""MultiPoint"",""coordinates"":[]}",1984
1,1_LT05_116034_19850311_797_0,278.949702,326.113118,LT05_116034_19850311,37.565193,126.987004,L5,1985-03-11 01:41:28,"{""type"":""MultiPoint"",""coordinates"":[]}",1985
2,1_LT05_116034_19860226_797_0,271.963270,326.113118,LT05_116034_19860226,37.565193,126.987004,L5,1986-02-26 01:37:29,"{""type"":""MultiPoint"",""coordinates"":[]}",1986
3,1_LT05_116034_19860415_797_0,300.356762,326.113118,LT05_116034_19860415,37.565193,126.987004,L5,1986-04-15 01:36:29,"{""type"":""MultiPoint"",""coordinates"":[]}",1986
4,1_LT05_116034_19860517_797_0,302.188820,326.113118,LT05_116034_19860517,37.565193,126.987004,L5,1986-05-17 01:35:40,"{""type"":""MultiPoint"",""coordinates"":[]}",1986
...,...,...,...,...,...,...,...,...,...,...
2644639,3_LC08_116034_20251104_4458_0,290.868338,325.944916,LC08_116034_20251104,37.574882,127.041636,L8,2025-11-04 02:11:21,"{""type"":""MultiPoint"",""coordinates"":[]}",2025
2644640,4_LC09_116034_20251112_4458_0,290.212078,325.944916,LC09_116034_20251112,37.574882,127.041636,L9,2025-11-12 02:11:26,"{""type"":""MultiPoint"",""coordinates"":[]}",2025
2644641,4_LC09_116034_20251128_4458_0,281.089383,325.944916,LC09_116034_20251128,37.574882,127.041636,L9,2025-11-28 02:11:24,"{""type"":""MultiPoint"",""coordinates"":[]}",2025
2644642,3_LC08_116034_20251222_4458_0,274.424244,325.944916,LC08_116034_20251222,37.574882,127.041636,L8,2025-12-22 02:11:18,"{""type"":""MultiPoint"",""coordinates"":[]}",2025


In [ ]:
export_csv_copy.to_csv("/content/sample_data/export_clean.csv", index=False)